In [1]:
# Libraries
import open3d as o3d
import viser
import torch
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
#@title Importing the Clusters with It's radiances
_3DBbox = torch.load("/workspace/Desktop/FruitProposal/attachment/RadianceCloud/fruit_proposal_clusters_output.pt")

print("Number of Clusters: ", len(_3DBbox["cluster_center"]))
print(_3DBbox.keys())

n_cluster = 30

FileNotFoundError: [Errno 2] No such file or directory: '/workspace/FruitProposal/attachment/RadianceCloud/fruit_proposal_clusters_output.pt'

In [ ]:
#TEST
# Obtaining the graph nodes
corners       = _3DBbox["cluster_corners"][n_cluster]
upper_corners = _3DBbox["cluster_u_corners"][n_cluster]
lower_corners = _3DBbox["cluster_l_corners"][n_cluster]
center        = _3DBbox["cluster_center"][n_cluster]
radius        = _3DBbox["cluster_radius"][n_cluster]

radiance = _3DBbox["radiance"]
points   = radiance["points"][n_cluster]
rgb      = radiance["rgb"][n_cluster]

v1 = corners[0]
v2 = corners[1]
v3 = corners[2]
v4 = corners[3]

# Compute edge vectors
e1 = v2 - v1
e2 = v3 - v1
e3 = v4 - v1

print("Edge vectors:")
print("e1:", e1)
print("e2:", e2)
print("e3:", e3)

# Normalize to get unit directions
u1 = e1 / np.linalg.norm(e1)
u2 = e2 / np.linalg.norm(e2)
u3 = e3 / np.linalg.norm(e3)

# Build local-to-world rotation matrix
T = np.eye(3,3)

# Propagate a 3x3x3 grid in local coordinates into world space
edge_length = np.linalg.norm(e1)  # assume all edges equal length
grid_size = 20
points_world = []
for i in range(grid_size):
    for j in range(grid_size):
        for k in range(grid_size):
            local_pt = np.array([i, j, k]) * (edge_length / (grid_size - 1))
            world_pt = v1 + T @ local_pt
            points_world.append(world_pt)
points_world = np.array(points_world)

Edge vectors:
e1: [0.05194551 0.         0.        ]
e2: [0.         0.05194551 0.        ]
e3: [0.        0.        0.0519455]


In [ ]:
pcd_c_grids = o3d.geometry.PointCloud()
pcd_c_grids.points = o3d.utility.Vector3dVector(points_world)
pcd_c_grids.paint_uniform_color([0.5, 0.5, 0.5])  # Gray color for grids
pcd_c_grids.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# Visualize the point cloud with grids using viser
server = viser.ViserServer()

# Grids
server.scene.add_point_cloud(
    name="/grid_points",
    points=np.asarray(pcd_c_grids.points),
    colors=np.asarray(pcd_c_grids.colors),
    point_size=0.0002,
    visible=False
)

# Bbox
server.scene.add_box(
    name=f"/bbox_cube/cluster_0_box",
    position=center,
    dimensions=np.array([radius * 2, radius * 2, radius * 2]),
    color=np.array([1.0, 0.0, 0.0]),  # black
    visible=False
)


╭─────────────── viser ───────────────╮
│             ╷                       │
│   HTTP      │ http://0.0.0.0:8080   │
│   Websocket │ ws://0.0.0.0:8080     │
│             ╵                       │
╰─────────────────────────────────────╯

MeshHandle(_impl=_SceneNodeHandleState(name='/bbox_cube/cluster_0_box', api=<viser._scene_api.SceneApi object at 0x7fcfe95356c0>, wxyz=array([1., 0., 0., 0.]), position=array([ 0.27118844,  0.2108236 , -0.03763865], dtype=float32), visible=False, click_cb=None))

In [ ]:
# Definitive Slices of the Bbox
upper_corners = _3DBbox["cluster_u_corners"][n_cluster]
lower_corners = _3DBbox["cluster_l_corners"][n_cluster]
radius        = _3DBbox["cluster_radius"][n_cluster]

In [ ]:
# Upper Corners
upper_1 = upper_corners[0]
upper_2 = upper_corners[1]
upper_3 = upper_corners[2]
upper_4 = upper_corners[3]

# Lower Corners
lower_1 = lower_corners[0]
lower_2 = lower_corners[1]
lower_3 = lower_corners[2]
lower_4 = lower_corners[3]

In [ ]:
# Edge Vectors for upper corners to Lower corners
e1 = upper_1 - lower_1
e2 = upper_2 - lower_2
e3 = upper_3 - lower_3
e4 = upper_4 - lower_4

print("Edge vectors for upper corners to lower corners:")
print("e1:", e1)
print("e2:", e2)
print("e3:", e3)
print("e4:", e4)

Edge vectors for upper corners to lower corners:
e1: [0.        0.        0.0519455]
e2: [0.        0.        0.0519455]
e3: [0.        0.        0.0519455]
e4: [0.        0.        0.0519455]


In [ ]:
# Propagate a 3x3x3 grid in local coordinates into world space
edge_length = np.linalg.norm(e1)  # assume all edges equal length
grid_size = 10 #50
t_vals = np.linspace(0, 1, grid_size)

# Generate slice planes and sub-bboxes
subboxes = []
for k in range(grid_size - 1):
    t0, t1 = t_vals[k], t_vals[k + 1]
    top_slice = upper_corners + (lower_corners - upper_corners) * t0
    bottom_slice = upper_corners + (lower_corners - upper_corners) * t1
    # Combine top and bottom for 8 corners
    bbox_corners = np.vstack([top_slice, bottom_slice])
    subboxes.append(bbox_corners)

    pcd_slice = o3d.geometry.PointCloud()
    pcd_slice.points = o3d.utility.Vector3dVector(bbox_corners)
    pcd_slice.paint_uniform_color([1, 0, 0])  # red color for grids
    pcd_slice.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    # Grids
    server.scene.add_point_cloud(
        name="/slice/slice_{}_points".format(k),
        points=np.asarray(pcd_slice.points),
        colors=np.asarray(pcd_slice.colors),
        point_size=0.0002,
        visible=True
    )

In [ ]:
radiance = _3DBbox["radiance"]
points   = radiance["points"][n_cluster]
rgb      = radiance["rgb"][n_cluster]

points = points.cpu().numpy()
rgb = rgb.cpu().numpy()

# Filtering Points based on the sub-bboxes
def filter_points_in_bbox(points, bbox_corners):
    min_corner = np.min(bbox_corners, axis=0)
    max_corner = np.max(bbox_corners, axis=0)
    mask = np.all((points >= min_corner) & (points <= max_corner), axis=1)
    return points[mask]

_slice = {}
_points = []
_plane = []
_rgb = []
_slice_corner = []
_slice_center = []
_max_points = np.max([len(x) for x in radiance["points"]])
print("Max points: ", _max_points)

# Visualize filtered points for each sub-bbox
for k, bbox_corners in enumerate(subboxes):
    filtered_points = filter_points_in_bbox(points, bbox_corners)
    filtered_rgb = rgb[np.all((points >= np.min(bbox_corners, axis=0)) & (points <= np.max(bbox_corners, axis=0)), axis=1)]

    _rgb.append(filtered_rgb)
    _points.append(filtered_points)

    pcd_filtered = o3d.geometry.PointCloud()
    pcd_filtered.points = o3d.utility.Vector3dVector(filtered_points)
    pcd_filtered.colors = o3d.utility.Vector3dVector(filtered_rgb)

    server.scene.add_point_cloud(
        name="/filtered/slice_{}".format(k),
        points=np.asarray(pcd_filtered.points),
        colors=np.asarray(pcd_filtered.colors),
        point_size=0.0005,
        visible=True
    )

    # Modify Z of each point to match the slice level
    z_level = (bbox_corners[0, 2] + bbox_corners[1, 2]) / 2
    filtered_points[:, 2] = z_level

    # Create slice_corners and transform them to the correct level
    slice_corners = bbox_corners.copy()
    slice_corners[:, 2] = z_level
    slice_corners = np.array([slice_corners[0], slice_corners[1], slice_corners[2], slice_corners[4]])

    _slice_corner.append(slice_corners)
    _slice_center.append(np.mean(slice_corners, axis=0))

    _plane.append(filtered_points)

    pcd_flatten= o3d.geometry.PointCloud()
    pcd_flatten.points = o3d.utility.Vector3dVector(filtered_points)
    pcd_flatten.colors = o3d.utility.Vector3dVector(filtered_rgb)

    server.scene.add_point_cloud(
        name="/flatten/slice_{}".format(k),
        points=np.asarray(pcd_flatten.points),
        colors=np.asarray(pcd_flatten.colors),
        point_size=0.0005,
        visible=True
    )

_slice["points"]       = np.array(_points, dtype=object)
_slice["plane"]        = np.array(_plane, dtype=object)
_slice["rgb"]          = np.array(_rgb, dtype=object)
_slice["slice_corner"] = np.array(_slice_corner)
_slice["slice_center"] = np.array(_slice_center)

torch.save(_slice, "/workspace/FruitProposal/attachment/RadianceCloud/slice_points.pt")

while True:
    x = input("Press Enter to continue...")
    if x == "":
        break

Max points:  9819


(viser) Connection opened (0, 1 total), 480 persistent messages